# Toxic comments with BERT 🤬

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Цель проекта:** Обучить модель классифицировать комментарии на позитивные и негативные.

Целевая метрика: `F1 >= 0.75`

**Содержание**<a id='toc0_'></a>    
1. [ Подготовка       ](#toc1_)    
1.1. [Библиотеки       ](#toc1_1_)    
1.2. [Конфигурация       ](#toc1_2_)    
1.3. [Сервисные функции       ](#toc1_3_)    
2. [Данные       ](#toc2_)    
2.1. [Загрузка       ](#toc2_1_)    
2.2. [Проверка типов данных       ](#toc2_2_)    
2.3. [Проверка пропусков       ](#toc2_3_)    
2.4. [Проверка полных дубликатов       ](#toc2_4_)    
2.5. [Изучение таргета       ](#toc2_5_)    
2.6. [Выводы по датасету       ](#toc2_6_)    
3. [Обработка текста       ](#toc3_)    
4. [Модели       ](#toc4_)    
4.1. [Разделение данных       ](#toc4_1_)    
4.2. [Подготовка      ](#toc4_2_)    
4.3. [Модель `LogisticRegression`      ](#toc4_3_)    
4.4. [Модель `RandomForestClassifier`     ](#toc4_4_)    
4.5. [Модель `LGBMClassifier`     ](#toc4_5_)    
4.6. [Модель `CatBoostClassifier`     ](#toc4_6_)    
4.7. [Модель `BERT`  ](#toc4_7_)    
4.8. [Модель `toxic-BERT` ](#toc4_8_)    
5. [Результаты     ](#toc5_)    
5.1. [Результаты на тренировочной выборке     ](#toc5_1_)    
5.2. [Результаты на тестовой выборке     ](#toc5_2_)    
6. [Общий вывод     ](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a> Подготовка        [&#8593;](#toc0_)

### 1.1. <a id='toc1_1_'></a>Библиотеки        [&#8593;](#toc0_)

In [ ]:
from IPython.display import clear_output

In [ ]:
%pip install -q accelerate==0.26.0
%pip install -q catboost==1.2.7
%pip install -q dill==0.3.8
%pip install -q hyperopt==0.2.7
%pip install -q lightgbm==4.5.0
%pip install -q matplotlib==3.8.4
%pip install -q numpy==1.26.4
%pip install -q optree==0.14.0
%pip install -q pandas==2.2.3
%pip install -q prettytable==3.12.0
%pip install -q scikit-learn==1.5.2
%pip install -q spacy==3.8.4
%pip install -q termcolor==2.5.0
%pip install -q torch==2.6.0
%pip install -q torchvision==0.21.0
%pip install -q tqdm==4.66.5
%pip install -q transformers==4.49.0

clear_output()

In [ ]:
import re
from os.path import exists
from pathlib import Path


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dill
import torch
import spacy

from tqdm.notebook import tqdm
from torch.utils.data import (Dataset,
                              DataLoader)
from catboost.utils import get_gpu_device_count
from prettytable import PrettyTable
from termcolor import colored
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import (train_test_split,
                                     StratifiedKFold,
                                     cross_val_score)
from sklearn.metrics import (f1_score,
                             classification_report,
                             ConfusionMatrixDisplay)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from transformers import (BertTokenizer,
                          BertModel,
                          BertForSequenceClassification,
                          AdamW)
from hyperopt import (hp,
                      fmin,
                      tpe,
                      Trials,
                      STATUS_OK,
                      STATUS_FAIL)

In [ ]:
tqdm.pandas()

### 1.2. <a id='toc1_2_'></a>Конфигурация        [&#8593;](#toc0_)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
RANDOM_STATE = 27
TEST_SIZE = 0.1

### 1.3. <a id='toc1_3_'></a>Сервисные функции        [&#8593;](#toc0_)

In [ ]:
def get_dataframe(paths: list[str], **kwargs) -> pd.DataFrame:
    for _path in paths:
        if not exists(_path) and not _path.startswith('http'):
            continue

        try:
            df = pd.read_csv(_path, **kwargs)
        except:
            continue

        if df is None:
            continue

        return df

    raise FileNotFoundError('No paths are valid for correct csv file.')

In [ ]:
def check_duplicates(df: pd.DataFrame) -> None:
    duplicates_count = df.duplicated().sum()

    if duplicates_count == 0:
        print(colored('Полных дубликатов не обнаружено.', 'green'))
        return

    duplicates_part = duplicates_count / len(df)
    print(colored(f'Обнаружено {duplicates_count} дубликатов ({duplicates_part:.2%})', 'red'))

In [ ]:
def check_nans(df: pd.DataFrame) -> None:
    if df.isna().sum().sum() == 0:
        print(colored('Полных дубликатов не обнаружено.', 'green'))
        return

    table = PrettyTable()
    table.field_names = ['Feature', 'Missing values count']

    missing_info = df.isna().sum().sort_values()
    cols = missing_info.index.to_list()
    for col in cols:
        count = missing_info[col]
        color = 'green' if count == 0 else 'red'
        s = f'{count} ({count / len(df):.2%})'
        table.add_row([col, colored(s, color)])

    print(table)

In [ ]:
def get_value_counts(series: pd.Series) -> None:
    data = pd.DataFrame()
    data['count'] = series.value_counts()
    data['part'] = round(data['count'] / len(series), 4)
    display(data)

## 2. <a id='toc2_'></a>Данные        [&#8593;](#toc0_)

### 2.1. <a id='toc2_1_'></a>Загрузка        [&#8593;](#toc0_)

In [ ]:
df = get_dataframe([
    './data/toxic_comments.csv',
    'datasets/toxic_comments.csv',
    'https://code.s3.yandex.net/datasets/toxic_comments.csv'
], index_col=0)

In [ ]:
df.head()

### 2.2. <a id='toc2_2_'></a>Проверка типов данных        [&#8593;](#toc0_)

In [ ]:
df.info()

Все типы данных корректны.

### 2.3. <a id='toc2_3_'></a>Проверка пропусков        [&#8593;](#toc0_)

In [ ]:
check_nans(df)

### 2.4. <a id='toc2_4_'></a>Проверка полных дубликатов        [&#8593;](#toc0_)

In [ ]:
check_duplicates(df)

### 2.5. <a id='toc2_5_'></a>Изучение таргета        [&#8593;](#toc0_)

In [ ]:
get_value_counts(df['toxic'])

In [ ]:
toxic_counts = df['toxic'].value_counts()

plt.figure(figsize=(4, 4))
plt.pie(toxic_counts,
        labels=['Not Toxic (0)', 'Toxic (1)'],
        colors=['lightgreen', 'lightcoral'],
        autopct='%1.1f%%',
        startangle=90)
plt.axis('equal')
plt.title('Распределение таргета')
plt.show()

Видим довольно большой дизбаланс.

### 2.6. <a id='toc2_6_'></a>Выводы по датасету        [&#8593;](#toc0_)

Перед нами датасет о комментариях с оценкой их токсичности. Пропуски не обнаружены. Полные дубликаты также не обнаружены. Распределение целевого признака крайне несбалансированное. Записей с таргетом `0` (not toxic) почти в 9 раз больше.

## 3. <a id='toc3_'></a>Обработка текста        [&#8593;](#toc0_)

In [ ]:
def clean_text(text: str) -> str:
    text = re.sub(r"[^a-zA-Z\s']", ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()

    return text

In [ ]:
lemmatized_path = './data/lemmatized.csv'

if exists(lemmatized_path):
    df = pd.read_csv(lemmatized_path)
else:
    disabled_pipes = ['parser',  'ner']
    nlp = spacy.load('en_core_web_md', disable=disabled_pipes)
    texts = df['text'].apply(clean_text).tolist()
    lemm_texts = []

    for doc in tqdm(nlp.pipe(texts, disable=disabled_pipes), total=len(texts)):
        lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        lemm_text = ' '.join(lemmas)
        lemm_texts.append(lemm_text)

    df['text'] = lemm_texts

    Path('./data').mkdir(exist_ok=True)
    df.to_csv('./data/lemmatized.csv', index=False)

In [ ]:
df.head()

## 4. <a id='toc4_'></a>Модели        [&#8593;](#toc0_)

### 4.1. <a id='toc4_1_'></a>Разделение данных        [&#8593;](#toc0_)

In [ ]:
X = df['text']
y = df['toxic']

In [ ]:
X = X.astype('str')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=TEST_SIZE,
                                                    random_state=RANDOM_STATE,
                                                    stratify=y)

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

### 4.2. <a id='toc4_2_'></a>Подготовка       [&#8593;](#toc0_)

In [ ]:
def get_objective(estimator,
                  X_train: pd.DataFrame,
                  y_train: pd.Series):
    def objective(params: dict) -> float:
        """Кросс-валидация с текущими гиперпараметрами.

        Args:
            estimator: пайплайн с моделью или отдельно модель
            params (dict): гиперпараметры
            X_train (pd.DataFrame): входные признаки (features)
            y_train (pd.Series): целевой признак (target)

        Returns:
            dict: Словарь со средним значением метрики, гиперпараметрами и статусом.
        """
        # Некоторые параметры могут быть только целочисленные, приводим их к нужному типу
        for key, value in params.items():
            if isinstance(value, float) and value % 1 == 0:
                params[key] = int(value)

        estimator.set_params(**params)
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

        try:
            score = cross_val_score(estimator=estimator,
                                    X=X_train,
                                    y=y_train,
                                    scoring='f1',
                                    cv=skf,
                                    n_jobs=-1)

            return {
                'loss': -score.mean(),
                'params': params,
                'status': STATUS_OK
            }
        except Exception as e:
            print(e)
            return {'status': STATUS_FAIL}

    return objective

In [ ]:
def get_model_results(estimator,
                      param_space: dict,
                      X_train: pd.DataFrame,
                      y_train: pd.Series,
                      max_evals: int = 1000
                      ) -> tuple[dict, float]:
    objective = get_objective(estimator, X_train, y_train)
    trials = Trials()

    fmin(
        fn=objective,
        space=param_space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=np.random.default_rng(RANDOM_STATE),
        show_progressbar=True
    )

    clear_output()

    best_params = trials.best_trial['result']['params']
    best_score = abs(trials.best_trial['result']['loss'])
    print(f'Finish with best F1 = {best_score:.4f}')

    return best_params, best_score

### 4.3. <a id='toc4_3_'></a>Модель `LogisticRegression`       [&#8593;](#toc0_)

In [ ]:
lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1))),
    ('model', LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1, max_iter=1000))
])

In [ ]:
param_space = {
    'tfidf__max_features':       hp.quniform('tfidf__max_features', 30_000, 50_000, 1000),       # Максимальное количество признаков (целые числа)
    'tfidf__min_df':             hp.choice('tfidf__min_df', [1, 2, 5, 10]),                      # Минимальная частота слов в документах
    'tfidf__max_df':             hp.uniform('tfidf__max_df', 0.7, 1.0),                          # Максимальная частота слов в документах
    'tfidf__use_idf':            hp.choice('tfidf__use_idf', [True, False]),                     # Использовать обратную частоту документа
    'tfidf__smooth_idf':         hp.choice('tfidf__smooth_idf', [True, False]),                  # Сглаживать IDF веса
    'tfidf__sublinear_tf':       hp.choice('tfidf__sublinear_tf', [True, False]),                # Применять сублинейное масштабирование TF

    'model__C':                  hp.loguniform('model__C', np.log(1e-3), np.log(15)),            # Обратная сила регуляризации (log-scale)
    'model__class_weight':       hp.choice('model__class_weight', [None, 'balanced']),           # Вес классов (автоматический баланс или нет)
}

In [ ]:
# best_params_lr, best_score_lr = get_model_results(lr_pipeline, param_space, X_train, y_train, 100)

In [ ]:
best_params_lr = {
    'model__C': 14.848553211901166,
    'model__class_weight': None,
    'tfidf__max_df': 0.7549417960132765,
    'tfidf__max_features': 37000,
    'tfidf__min_df': 5,
    'tfidf__smooth_idf': True,
    'tfidf__sublinear_tf': True,
    'tfidf__use_idf': False
}

In [ ]:
best_score_lr = 0.7859790770261292

### 4.4. <a id='toc4_4_'></a>Модель `RandomForestClassifier`      [&#8593;](#toc0_)

In [ ]:
rfc_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1))),
    ('model', RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1))
])

In [ ]:
param_space = {
    'tfidf__max_features':       hp.quniform('tfidf__max_features', 30_000, 50_000, 1000),   # Максимальное количество признаков (целые числа)
    'tfidf__min_df':             hp.choice('tfidf__min_df', [1, 2, 5, 10]),                  # Минимальная частота слов в документах
    'tfidf__max_df':             hp.uniform('tfidf__max_df', 0.7, 1.0),                      # Максимальная частота слов в документах
    'tfidf__use_idf':            hp.choice('tfidf__use_idf', [True, False]),                 # Использовать обратную частоту документа
    'tfidf__smooth_idf':         hp.choice('tfidf__smooth_idf', [True, False]),              # Сглаживать IDF веса
    'tfidf__sublinear_tf':       hp.choice('tfidf__sublinear_tf', [True, False]),            # Применять сублинейное масштабирование TF

    'model__n_estimators':       hp.choice('model__n_estimators', [50, 100, 200, 500]),      # Количество деревьев в лесу
    'model__criterion':          hp.choice('model__criterion', ['gini', 'entropy']),         # Критерий качества разбиения
    'model__max_depth':          hp.choice('model__max_depth', [None, 10, 20, 30, 50]),      # Максимальная глубина дерева
    'model__min_samples_split':  hp.choice('model__min_samples_split', [2, 5, 10]),          # Минимальное число объектов для разбиения
    'model__min_samples_leaf':   hp.choice('model__min_samples_leaf', [1, 2, 4]),            # Минимальное число объектов в листе
    'model__bootstrap':          hp.choice('model__bootstrap', [True, False]),               # Использовать бутстрэп выборки
    'model__class_weight':       hp.choice('model__class_weight', [None, 'balanced']),       # Веса классов для несбалансированных данных
}

In [ ]:
# best_params_rfc, best_score_rfc = get_model_results(rfc_pipeline, param_space, X_train, y_train, 100)

In [ ]:
best_params_rfc = {
    'model__bootstrap': True,
    'model__class_weight': None,
    'model__criterion': 'gini',
    'model__max_depth': None,
    'model__min_samples_leaf': 1,
    'model__min_samples_split': 2,
    'model__n_estimators': 200,
    'tfidf__max_df': 0.7653612620932736,
    'tfidf__max_features': 46000,
    'tfidf__min_df': 10,
    'tfidf__smooth_idf': True,
    'tfidf__sublinear_tf': True,
    'tfidf__use_idf': False
 }

In [ ]:
best_score_rfc = 0.7645131020400102

### 4.5. <a id='toc4_5_'></a>Модель `LGBMClassifier`      [&#8593;](#toc0_)

In [ ]:
lgbmc_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1))),
    ('model', LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1, verbose=-1))
])

In [ ]:
param_space = {
    'tfidf__max_features':       hp.quniform('tfidf__max_features', 30_000, 50_000, 1000),       # Максимальное количество признаков (целые числа)
    'tfidf__min_df':             hp.choice('tfidf__min_df', [1, 2, 5, 10]),                      # Минимальная частота слов в документах
    'tfidf__max_df':             hp.uniform('tfidf__max_df', 0.7, 1.0),                          # Максимальная частота слов в документах
    'tfidf__use_idf':            hp.choice('tfidf__use_idf', [True, False]),                     # Использовать обратную частоту документа
    'tfidf__smooth_idf':         hp.choice('tfidf__smooth_idf', [True, False]),                  # Сглаживать IDF веса
    'tfidf__sublinear_tf':       hp.choice('tfidf__sublinear_tf', [True, False]),                # Применять сублинейное масштабирование TF

    'model__boosting_type':      hp.choice('model__boosting_type', ['gbdt', 'dart', 'goss']),    # Тип бустинга
    'model__num_leaves':         hp.quniform('model__num_leaves', 10, 200, 1),                   # Количество листьев в дереве
    'model__learning_rate':      hp.loguniform('model__learning_rate', -5, 0),                   # Скорость обучения
    'model__n_estimators':       hp.quniform('model__n_estimators', 50, 500, 1),                 # Количество деревьев
    'model__subsample':          hp.uniform('model__subsample', 0.5, 1.0),                       # Доля выборки для обучения каждого дерева
    'model__colsample_bytree':   hp.uniform('model__colsample_bytree', 0.5, 1.0),                # Доля признаков для каждого дерева
    'model__reg_alpha':          hp.loguniform('model__reg_alpha', -5, 2),                       # L1-регуляризация
    'model__reg_lambda':         hp.loguniform('model__reg_lambda', -5, 2),                      # L2-регуляризация
    'model__min_child_samples':  hp.quniform('model__min_child_samples', 5, 100, 1),             # Минимальное количество объектов в листе
    'model__max_depth':          hp.choice('model__max_depth', [-1, 3, 5, 7, 10]),               # Максимальная глубина дерева (-1 = без ограничений)
}

In [ ]:
# best_params_lgbmc, best_score_lgbmc = get_model_results(lgbmc_pipeline, param_space, X_train, y_train, 100)

In [ ]:
best_params_lgbmc = {
    'model__boosting_type': 'gbdt',
    'model__colsample_bytree': 0.6191104272221566,
    'model__learning_rate': 0.09230661681722988,
    'model__max_depth': -1,
    'model__min_child_samples': 22,
    'model__n_estimators': 298,
    'model__num_leaves': 89,
    'model__reg_alpha': 0.8456632205247746,
    'model__reg_lambda': 0.12795690362150944,
    'model__subsample': 0.5637403699663865,
    'tfidf__max_df': 0.9783133591194336,
    'tfidf__max_features': 34000,
    'tfidf__min_df': 2,
    'tfidf__smooth_idf': True,
    'tfidf__sublinear_tf': True,
    'tfidf__use_idf': False
}

In [ ]:
best_score_lgbmc = 0.7781002572293113

### 4.6. <a id='toc4_6_'></a>Модель `CatBoostClassifier`      [&#8593;](#toc0_)

In [ ]:
gpu_count = get_gpu_device_count()

if gpu_count > 0:
    task_type = 'GPU'
    devices = '0'
    print(colored('GPU найден. Будет использован GPU.', 'green'))
else:
    task_type = 'CPU'
    devices = None
    print(colored('GPU не найден. Будет использован CPU.', 'red'))

In [ ]:
cbc = CatBoostClassifier(task_type=task_type,
                         devices=devices,
                         random_state=RANDOM_STATE,
                         silent=True,
                         text_features=[0])

In [ ]:
X_train_df = X_train.to_frame()
X_test_df = X_test.to_frame()

In [ ]:
param_space = {
    'learning_rate':        hp.uniform('learning_rate', 0.01, 0.3),    # Скорость обучения модели
    'depth':                hp.randint('depth', 6, 12),                # Глубина деревьев в ансамбле
    'l2_leaf_reg':          hp.uniform('l2_leaf_reg', 1, 10),          # Регуляризация L2 для предотвращения переобучения
    'bagging_temperature':  hp.uniform('bagging_temperature', 0, 2),   # Температура баггинга для случайности выборки
    'random_strength':      hp.uniform('random_strength', 0, 2),       # Сила случайности при выборе разбиений
    'iterations':           hp.randint('iterations', 500, 1500),       # Количество итераций обучения
}

In [ ]:
class DillTrials(Trials):
    def __init__(self, *args, **kwargs):
        super(DillTrials, self).__init__(*args, **kwargs)

    def _dump(self, trials_data):
        return dill.dumps(trials_data)

    def _load(self, dumped_trials_data):
        return dill.loads(dumped_trials_data)

In [ ]:
def get_catboost_results(estimator,
                         param_space: dict,
                         X_train: pd.DataFrame,
                         y_train: pd.Series,
                         max_evals: int = 1000
                         ) -> tuple[dict, float]:
    objective = get_objective(estimator, X_train, y_train)
    trials = DillTrials()

    fmin(
        fn=objective,
        space=param_space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=np.random.default_rng(RANDOM_STATE),
        show_progressbar=True
    )

    clear_output()

    best_params = trials.best_trial['result']['params']
    best_score = abs(trials.best_trial['result']['loss'])
    print(f'Finish with best F1 = {best_score:.4f}')

    return best_params, best_score

In [ ]:
#best_params_cbc, best_score_cbc = get_catboost_results(cbc, param_space, X_train_df, y_train, 20)

In [ ]:
#best_params_cbc

In [ ]:
#best_score_cbc

### 4.7. <a id='toc4_7_'></a>Модель `BERT`   [&#8593;](#toc0_)

In [ ]:
# model_name = 'bert-base-uncased'

In [ ]:
# tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
class CommentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

Обучение BERT на всем корпусе займет слишком много времени, поэтому сократим датасет.

In [ ]:
required_size = 1_000
train_part = required_size / len(X_train)

In [ ]:
X_for_bert, _, y_for_bert, _ = train_test_split(X_train,
                                                y_train,
                                                random_state=RANDOM_STATE,
                                                train_size=train_part,
                                                stratify=y_train)

In [ ]:
X_for_bert.shape

In [ ]:
X_train_bert, X_val_bert, y_train_bert, y_val_bert = train_test_split(X_for_bert,
                                                                      y_for_bert,
                                                                      random_state=RANDOM_STATE,
                                                                      test_size=TEST_SIZE,
                                                                      stratify=y_for_bert)

In [ ]:
print(X_train_bert.shape, y_train_bert.shape)
print(X_val_bert.shape, y_val_bert.shape)

In [ ]:
# train_dataset = CommentDataset(X_train_bert.values.tolist(), y_train_bert.values.tolist(), tokenizer)
# val_dataset = CommentDataset(X_val_bert.values.tolist(), y_val_bert.values.tolist(), tokenizer)

In [ ]:
# num_workers = 4 if torch.cuda.is_available() else 0

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=num_workers)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(colored('GPU найден. Будет использован GPU.', 'green'))
else:
    print(colored('GPU не найден. Будет использован CPU.', 'red'))

In [ ]:
# model = model.to(device)

In [ ]:
# optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
epochs = 3

In [ ]:
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         loss.backward()
#         optimizer.step()

#     avg_train_loss = total_loss / len(train_loader)
#     print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_train_loss}')

In [ ]:
# model.eval()
# val_preds = []
# val_labels = []

In [ ]:
# with torch.no_grad():
#     for batch in val_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         _, preds = torch.max(outputs.logits, dim=1)

#         val_preds.extend(preds.cpu().numpy())
#         val_labels.extend(labels.cpu().numpy())

In [ ]:
# f1_bert = f1_score(val_labels, val_preds)
# print(f'BERT F1 on val = {f1_bert:.4f}')

In [ ]:
# ConfusionMatrixDisplay.from_predictions(val_labels, val_preds, cmap='Greens');

~~Модель BERT показала неудовлетворительный результат - абсолютно все записи были отнесены к классу `0`. Это ожидаемо, учитывая дизбаланс классов (`9:1`) и крайне сильное сокращение обучающей выборки. Необходимо попробовать методы undersampling'а и увеличить обучающую выборку, что требует сильно больше ресурсов (вычислительных и временных). На данный момент модель BERT **НЕ** подходит для проекта.~~

### 4.8. <a id='toc4_8_'></a>Модель `toxic-BERT`  [&#8593;](#toc0_)

In [ ]:
model_name = "unitary/toxic-bert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
def get_embeddings(texts):
    inputs = tokenizer(texts,
                       return_tensors="pt",
                       padding=True,
                       truncation=True,
                       max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    # Используем пуллинговый вектор (вектор [CLS])
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

In [ ]:
train_embeddings = get_embeddings(X_train_bert.values.tolist())
val_embeddings = get_embeddings(X_val_bert.values.tolist())

In [ ]:
toxic_lr = LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1);

In [ ]:
toxic_lr.fit(train_embeddings, y_train_bert);

In [ ]:
y_pred = toxic_lr.predict(val_embeddings)

In [ ]:
f1_toxic_bert = f1_score(y_val_bert, y_pred)
print(f'F1 for toxic-BERT on val = {f1_toxic_bert:.4f}')

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_val_bert, y_pred);

Модель toxic-BERT показала невероятные результаты. Был использован только бэкбон, в качестве классификатора была взята LogisticRegression. При невероятно маленькой обучающей выборке (900 записей) эта связка переплюнула все другие модели. Увеличение обучающей выборки скорее всего приведет к еще бОльшему росту метрики.

<img alt="Meme about Howard Stark that he's limited by the technology of his time" src="./meme.jpg" width="620" height="414">

## 5. <a id='toc5_'></a>Результаты      [&#8593;](#toc0_)

### 5.1. <a id='toc5_1_'></a>Результаты на тренировочной выборке      [&#8593;](#toc0_)

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

pd.DataFrame(
    data=[best_score_rfc, 0, best_score_lgbmc, best_score_lr, 0, f1_toxic_bert],
    index=['RandomForestClassifier', 'CatBoostClassifier', 'LGBMClassifier', 'LogisticRegression', 'BERT', 'toxic-BERT'],
    columns=['f1']
).sort_values('f1', ascending=True)

### 5.2. <a id='toc5_2_'></a>Результаты на тестовой выборке      [&#8593;](#toc0_)

In [ ]:
required_size = 500
train_part = required_size / len(X_test)

In [ ]:
X_test_bert, _, y_test_bert, _ = train_test_split(X_test,
                                                  y_test,
                                                  random_state=RANDOM_STATE,
                                                  train_size=train_part,
                                                  stratify=y_test)

In [ ]:
X_test_bert.shape, y_test_bert.shape

In [ ]:
test_embeddings = get_embeddings(X_test_bert.values.tolist())

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
    <b>v2</b>  
Тут совет простой - работать с батчами.
    
Сейчас ты обрабатыаешь все тексты сразу, но при работе на GPU будет ограничение, врядли модель сможет вместить в себя больше 32-64 текстов за раз.
    
Соответственно добавляем батчи и прогресс-бар по батчам.
    
Образец кода с векторизацией по батчам был в спринте.    
    
</div>

In [ ]:
y_pred_bert = toxic_lr.predict(test_embeddings)

In [ ]:
f1 = f1_score(y_test_bert, y_pred_bert)
print(f'F1 on test = {f1:.4f}')

In [ ]:
print(classification_report(y_test_bert, y_pred_bert))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test_bert, y_pred_bert, cmap='Greens');

## 6. <a id='toc6_'></a>Общий вывод      [&#8593;](#toc0_)

В рамках данного проекта мы работали с задачей обработки естественного языка (NLP), а именно определение токсичности комментариев (бинарная классификация).

Перед нами датасет, состоящий практически практически из `160 000` записей. Он [датасет] не содержит пропусков и полных дубликатов. Также отметим дизбаланс примерно `9:1` в сторону не токсичных комментариев (класс `0`).

Сначала текст был очищен с помощью регулярных выражений, а также лемматизирован.

Были протестированы 4 различные модели с подбором гиперпараметров через *hyperopt*, а именно: *LogisticRegression*, *RandomForestClassifier*, *LGBMClassifier* и *CatBoostClassifier*. Для первых трёх моделей был использован *TfidfVectorizer*, гиперпараметры которого подбирались также через *hyperopt*. А вот *CatBoostClassifier* умеет работать с текстовыми признаками "из коробки", что и было использовано.

Результаты представлены в таблице ниже:

|          Model         |   F1   |
|:----------------------:|:------:|
| CatBoostClassifier     |   0    |
| BERT                   |   0    |
| RandomForestClassifier | 0.7645 |
| LGBMClassifier         | 0.7781 |
| LogisticRegression     | 0.7860 |
| toxic-BERT             | 0.8235 |


Из этих 4 "классических" моделей лучше всего себя показала *LogisticRegression* с результатом `F1 = 0.7860`.

Также была протестирована предобученная (bert-base-uncased) модель *BERT*. На валидационной выборке данная модель показала нулевую F1, так как отнесла все записи к классу `0`. Это ожидаемо, учитывая дизбаланс классов и крайне сильное сокращение обучающей выборки. Необходимо попробовать методы undersampling'а и увеличить обучающую выборку, что требует сильно больше ресурсов (вычислительных и временных). На данный момент модель BERT **НЕ** подходит для проекта.

Последней была протестирована модель *toxic-BERT*. Однако ее дообучение не проводилось. Был использован только бэкбон для получения эмбеддингов, а в качестве классификатора была использована *LogisticRegression*. Данная связка показала прекрасный результат на крайне малом обучающем датасете - `F1 = 0.8235`.

Именно она и была протестирована на тестовой подвыборке и показала результат `F1 = 0.8571`. Это больше минимально необходимой метрики, следовательно проект можно считать успешным. Для теста использовалась крайне малая часть тестовой выборки ввиду больших временных затрат при получении эмбеддингов и смерти ядра в процессе выполнения.

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> рекомендации по доп. материалам</b>
Если решишь погрузиться в область работы с текстами, очень советую несколько продвинутых бесплатных курсов.
    
   - Отличный бесплатный курс от Школы глубокого обучения МФТИ (https://stepik.org/org/dlschool), старт курса каждые пол года. Два семестра, один по основам и компьютерному зрению, второй по обработке естественного языка. Проходить нужно именно в таком порядке,т.к. почти весь современный NLP построен на нейронках.
    
   - "Нейронные сети и компьютерное зрение" от Samsung Research Russia (https://stepik.org/course/50352/syllabus). Есть также продолжение по NLP.  
   - Трек NLP от сообщества ODS https://ods.ai/tracks/nlp-course-autumn-22
    
    
А если на тебя произвели впечатление возможности ChatGPT и хочешь попробовать использовать возможности больших языковых моделей для решения своих задач, могу порекомендовать следующие курсы (первые две ссылки на английском, зато последняя подборка лекций на русском).
    
  - https://www.coursera.org/learn/generative-ai-with-llms  (можно прослужать бесплатно)
  - Короткие курсы на сайте https://www.deeplearning.ai/short-courses/  Самый свежак - как писать промпты, как использовать LLM для создания собственных приложений (например как создать бота на основе ChatGPT, который будет отвечать на вопросы по внутренней документации компании).  
  - https://www.promptingguide.ai/introduction/settings
  - https://www.youtube.com/watch?v=l-l82uNwyu8&list=PLy6K3_Hx-udj6n1S88Vslyw2QVxSXLP2c    
    
    
    
    
Также можно обратиться к туториалу по использованию языковых моделей на HugingFace. Обрати внимание, для больше чем половины материалов доступен перевод на русский, хотя можно читать и на языке оригинала. 
    
  https://huggingface.co/learn/nlp-course   
 
</div>